# Residual Networks

В рамках данной лабораторной работа разбирается принцип построения очень глубоких сверточных сетей, используя остаточные сети (Resnet). Теоретически очень глубокие сети могут представлять очень сложные функции, но на практике их трудно обучить. Остаточные сети, введенные [He et al.](http://arxiv.org/pdf/1512.03385.pdf), позволяют тренировать гораздо более глубокие сети, чем это было ранее практически осуществимо.

**В данной работе:**
- Реализуете основные строительные блоки ResNets. 
- Соберёте вместе эти строительные блоки, чтобы реализовать и обучить нейронную сеть для классификации изображений. 

`Данный материал опирается и использует материалы курса Deep Learning от организации deeplearning.ai`
 
 Ссылка на основной курс (для желающих получить сертификаты): https://www.coursera.org/specializations/deep-learning

In [ ]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

## 1 - Проблемы с очень глубокими нейронными сетями

В последние годы нейронные сети стали более глубокими, причем современные сети переходят от всего лишь нескольких слоев (например, AlexNet) к более чем сотне слоев.

* Главное преимущество очень глубокой сети заключается в том, что она может представлять очень сложные функции. Он также может изучать объекты на многих различных уровнях абстракции, от краев (на более мелких слоях, ближе к входу) до очень сложных объектов (на более глубоких слоях, ближе к выходу).
* Однако использование более глубокой сети не всегда помогает. Огромным препятствием для их обучения является исчезновение градиентов: очень глубокие сети часто имеют градиентный сигнал, который быстро идет к нулю, что делает градиентный спуск слишком медленным.
* Более конкретно, во время градиентного спуска, когда вы возвращаетесь от конечного слоя обратно к первому слою, вы умножаете весовую матрицу на каждом шаге, и таким образом градиент может быстро уменьшаться экспоненциально до нуля (или, в редких случаях, быстро расти экспоненциально и "взрываться", принимая очень большие значения).
* Таким образом, во время тренировки вы можете увидеть, что величина (или норма) градиента для более мелких слоев очень быстро уменьшается до нуля по мере продолжения тренировки:

<img src="images/vanishing_grad_kiank.png" style="width:450px;height:220px;">
<caption><center> <u> <font color='purple'>**Рисунок 1**</u><font color='purple'>  : **Исчезновение градиентов** <br> Скорость обучения снижается очень быстро для более мелких слоев по мере развития сети</center></caption>

## 2 - Построение Residual Network

В ResNets "shortcut" или "skip connection" позволяет модели пропускать слои:
<img src="images/skip_connection_kiank.png" style="width:650px;height:200px;">
<caption><center> <u> <font color='purple'>**Рисунок 2**</u><font color='purple'>  : ResNet блок - **skip-connection** <br> </center></caption>

Изображение слева показывает "основной путь" через сеть. Изображение справа добавляет ярлык к основному пути. Укладывая эти блоки ResNet друг на друга, вы можете сформировать очень глубокую сеть.

Наличие блоков ResNet с ярлыком очень облегчает для одного из блоков изучение функции идентификации. Это означает, что вы можете накапливать дополнительные блоки ResNet с небольшим риском ухудшения производительности обучающего набора.

В Сети ResNet используются два основных типа блоков в зависимости от того, одинаковы или различны размеры input/output: "identity block" и "convolutional block"

### 2.1 - Identity block (блок идентификации)

Блок идентификации это стандартный блок используемый в ResNet, и соответствует тому случаю, когда input activation ($a^{[l]}$) имеет такой же размер output activation ($a^{[l+2]}$). Чтобы конкретизировать различные шаги того, что происходит в блоке идентификации ResNet, вот альтернативная диаграмма, демонстрирующая отдельные шаги:
<img src="images/idblock2_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **Рисунок 3** </u><font color='purple'>  : **Identity block.** Пропуск соединения "пропускает" 2 слоя.</center></caption>

Верхний путь - это "кратчайший путь". Нижний путь- это "главный путь". На этой диаграмме представлены явными шаги CONV2D и ReLU в каждом слое. Чтобы ускорить обучение, мы также добавили шаг BatchNorm.

В этом упражнении необходимо реализовать несколько более мощную версию этого блока идентификации, в которой соединение пропуска 3 скрытых слоя, а не 2 слоя. Это выглядит примерно так:

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **Рисунок 4** </u><font color='purple'>  : **Identity block.** Пропускает 3 слоя.</center></caption>

Вот отдельные шаги.

Первый компонент - главный путь: 
- Первый CONV2D имеет $F_1$ фильтров размером (1,1) и шагом (1,1). Padding - "valid" и имя слоя корректно дать `conv_name_base + '2a'`.
- BatchNorm - нормализация 'channels' размерности.  Имя слоя - `bn_name_base + '2a'`.
- Затем ReLU activation function. Которая не имеет имя и гиперпараметры. 

Второй компонент - главный путь:
- Второй CONV2D имеет $F_2$ ильтров размером $(f,f)$ и шагом (1,1).  Padding - "same" и имя слоя корректно дать `conv_name_base + '2b'`.
- BatchNorm - нормализация 'channels' размерности.  Имя слоя - `bn_name_base + '2b'`.
- Затем ReLU activation function. Которая не имеет имя и гиперпараметры. 

Третий компонент - главный путь:
- Третий CONV2D имеет $F_3$ фильтров размером (1,1) и шагом (1,1). Padding - "valid" и имя слоя корректно дать `conv_name_base + '2c'`.
- BatchNorm - нормализация 'channels' размерности.  Имя слоя - `bn_name_base + '2c'`. 
- **Нет** ReLU activation function в данном компоненте. 

Конечный шаг: 
- `X_shortcut` и выход из 3-го слоя `X` сложенные вместе.
- **Подсказка**: Синтаксис `Add()([var1,var2])`
- Затем ReLU activation function. Которая не имеет имя и гиперпараметры. 

**Упражнение**: Реализовать ResNet identity block. Первый компонент главного пути уже реализован. Пожалуйста, прочтите внимательно, чтобы убедиться, что вы понимаете, что он делает. Вы должны реализовать все остальное.
- Реализовать Conv2D: [Conv2D](https://keras.io/layers/convolutional/#conv2d)
- Реализовать BatchNorm: [BatchNormalization](https://faroit.github.io/keras-docs/1.2.2/layers/normalization/)
- Реализовать функцию активации:  `Activation('relu')(X)`
- Сложить два входа: [Add](https://keras.io/layers/merge/#add)

In [ ]:
# ОЦЕНИВАЕМОЕ: identity_block

def identity_block(X, f, filters, stage, block):
    """
    Реализовать identity block представленный на Рисунке 4
    
    Arguments:
    X -- input tensor (m, n_H_prev, n_W_prev, n_C_prev)
    f -- целое число, задающее форму окна CONV для главного пути    
    filters -- python список integers, определяющих количество фильтров в слоях CONV основного пути
    stage -- integer, используется для наименования слоев в зависимости от их положения в сети
    block -- string/character, используется для наименования слоев в зависимости от их положения в сети
    
    Returns:
    X -- выход identity block, размером (n_H, n_W, n_C)
    """
    
    # определение пространства имён
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Извлечение фильтров
    F1, F2, F3 = filters
    
    # Сохраните входное значение. Это понадобится позже, чтобы добавить обратно в основной путь. 
    X_shortcut = X
    
    # Первый шаг - главный путь
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### НАЧАЛО ВАШЕГО КОД ###
    
    # Второй шаг главного пути (≈3 строки кода)
    X = None
    X = None
    X = None

    # Третьий шаг главного пути (≈2 строки кода)
    X = None
    X = None

    # Финальный шаг: Добавьте shortcut значение в главный путь и пропустите через RELU активацию (≈2 строки кода)
    X = None
    X = None
    
    ### ОКОНЧАНИЕ ВАШЕГО КОД ###
    
    return X

In [ ]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = identity_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

**Ожидаемый выход**:

<table>
    <tr>
        <td>
            **out**
        </td>
        <td>
           [ 0.94822985  0.          1.16101444  2.747859    0.          1.36677003]
        </td>
    </tr>

</table>

## 2.2 - Сonvolutional block (блок свёртки)

ResNet "сверточный блок" - это второй тип блока. Вы можете использовать этот тип блока, когда input и output размеры не совпадают. Разница с блоком идентификации заключается в том, что в кратчайшем пути есть CONV2Dlayer:

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **Рисунок 4** </u><font color='purple'>:**Convolutional block**</center></caption>

* Слой CONV2D в пути быстрого доступа используется для изменения размера входных данных $x$ в другое измерение, чтобы размеры совпадали в конечном добавлении, необходимом для добавления значения быстрого доступа обратно в основной путь.
* Например, чтобы уменьшить высоту и ширину размеров активации в 2 раза, можно использовать свертку 1x1 с шагом 2.
* Слой CONV2D на пути быстрого доступа не использует никакой нелинейной функции активации. Его основная роль заключается в том, чтобы просто применить линейную функцию, которая уменьшает размерность входных данных, так что размеры совпадают для последующего шага сложения.

Ключевые шаги следующие.

Первый компонент - главный путь: 
- Первый CONV2D имеет $F_1$ фильтров размером (1,1) и шагом (s,s). Padding - "valid" и имя слоя корректно дать `conv_name_base + '2a'`.
- BatchNorm - нормализация 'channels' размерности.  Имя слоя - `bn_name_base + '2a'`.
- Затем ReLU activation function. Которая не имеет имя и гиперпараметры.

Второй компонент - главный путь:
- Второй CONV2D имеет $F_2$ фильтров размером $(f,f)$ и шагом (1,1).  Padding - "same" и имя слоя корректно дать `conv_name_base + '2b'`.
- BatchNorm - нормализация 'channels' размерности.  Имя слоя - `bn_name_base + '2b'`.
- Затем ReLU activation function. Которая не имеет имя и гиперпараметры. 

Третий компонент - главный путь:
- Третий CONV2D имеет $F_3$ фильтров размером (1,1) и шагом (1,1). Padding - "valid" и имя слоя корректно дать `conv_name_base + '2c'`.
- BatchNorm - нормализация 'channels' размерности.  Имя слоя - `bn_name_base + '2c'`. 
- **Нет** ReLU activation function в данном компоненте.

Короткий путь:
- CONV2D имеет $F_3$ фильтров размером (1,1) и шагом (s,s). adding - "valid" и имя слоя корректно дать `conv_name_base + '1'`.  Use 0 as the `glorot_uniform` seed.
- The BatchNorm is normalizing the 'channels' axis.  Its name should be `bn_name_base + '1'`. 

Финальный шаг: 
- The shortcut and the main path values are added together.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 
    
**Упражнение**: Implement the convolutional block. We have implemented the first component of the main path; you should implement the rest. As before, always use 0 as the seed for the random initialization, to ensure consistency with our grader.
- [Conv2D](https://keras.io/layers/convolutional/#conv2d)
- [BatchNormalization](https://keras.io/layers/normalization/#batchnormalization) (axis: Integer, the axis that should be normalized (typically the features axis))
- For the activation, use:  `Activation('relu')(X)`
- [Add](https://keras.io/layers/merge/#add)

In [ ]:
# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = None
    X = None
    X = None

    # Third component of main path (≈2 lines)
    X = None
    X = None

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = None
    X_shortcut = None

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = None
    X = None
    
    ### END CODE HERE ###
    
    return X

In [ ]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = convolutional_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

**Expected Output**:

<table>
    <tr>
        <td>
            **out**
        </td>
        <td>
           [ 0.09018463  1.23489773  0.46822017  0.0367176   0.          0.65516603]
        </td>
    </tr>

</table>

## 3 - Построение ResNet модели (50 слоёв)

На следующем рисунке подробно описана архитектура этой нейронной сети. "ID BLOCK" на схеме означает "Identity block" и "ID BLOCK x3" означает, что вы должны объединить 3 тож блоков.

<img src="images/resnet_kiank.png" style="width:850px;height:150px;">
<caption><center> <u> <font color='purple'> **Рисунок 5** </u><font color='purple'>  : **ResNet-50 model** </center></caption>

Детальные шаги:
- Zero-padding с размером (3,3)
- Шаг 1:
    - 2D Convolution имеют 64 филтров с размером (7,7) и использует шаг (2,2). Имя - "conv1".
    - BatchNorm применяется к входным 'channels' в изображении.
    - MaxPooling использует window размером (3,3) и (2,2) шагом.
- Шаг 2:
    - Сonvolutional block использует три набора фильтров с размером [64,64,256], "f" - 3, "s" - 1 и блок - "a".
    - 2 identity blocks использует три набора фильтров с размером [64,64,256], "f" - 3 и блок - "b" и "c".
- Шаг 3:
    - Convolutional block использует три набора фильтров с размером [128,128,512], "f" - 3, "s" - 2 и блок - "a".
    - 3 identity blocks использует три набора фильтров с размером [128,128,512], "f" - 3 и блок - "b", "c" and "d".
- Шаг 4:
    - Convolutional block использует три набора фильтров с размером [256, 256, 1024], "f" - 3, "s" - 2 и блок - "a".
    - The 5 identity blocks использует три набора фильтров с размером [256, 256, 1024], "f" - 3 и блок - "b", "c", "d", "e" и "f".
- Шаг 5:
    - Convolutional block использует три набора фильтров с размером [512, 512, 2048], "f" - 3, "s" - 2 и блок - "a".
    - 2 identity blocks использует три набора фильтров с размером [512, 512, 2048], "f" - 3 и блок - "b" и "c".
- 2D Average Pooling использует окно с размером (2,2) и наименованием "avg_pool".
- 'flatten' слой не имеет гиперпараметров и имени.
- Fully Connected (Dense) слой уменьшается до количества классов. Его имя может быть `'fc' + str(classes)`.

**Упражнение**: Реализовать ResNet с 50 слоями описанными на рисунке выше.

Можно использовать следующие функции: 
- Average pooling [see reference](https://keras.io/layers/pooling/#averagepooling2d)
- Conv2D: [See reference](https://keras.io/layers/convolutional/#conv2d)
- BatchNorm: [See reference](https://keras.io/layers/normalization/#batchnormalization)
- Zero padding: [See reference](https://keras.io/layers/convolutional/#zeropadding2d)
- Max pooling: [See reference](https://keras.io/layers/pooling/#maxpooling2d)
- Fully connected layer: [See reference](https://keras.io/layers/core/#dense)
- Addition: [See reference](https://keras.io/layers/merge/#add)

In [ ]:
# ОЦЕНИВАЕМОЕ: ResNet50

def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Реализовать архитектуру ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- размер входных изображений из датасета
    classes -- integer, номер класса

    Returns:
    model -- Model() класс в Keras
    """
    
    # входной слой
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Шаг 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Шаг 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### НАЧАЛО ВАШЕГО КОДА ###

    # Шаг 3 (≈4 строки кода)
    X = None
    X = None
    X = None
    X = None

    # Шаг 4 (≈6 строки кода)
    X = None
    X = None
    X = None
    X = None
    X = None
    X = None

    # Шаг 5 (≈3 строки кода)
    X = None
    X = None
    X = None

    # AVGPOOL (≈1 строка кода) - "X = AveragePooling2D(...)(X)"
    X = None
    
    ### ОКОНЧАНИЕ ВАШЕГО КОДА ###

    # выходной слой
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # создание модели
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Модель готова к обучению.

Загрузка SIGNS Dataset.
<img src="images/signs_data_kiank.png" style="width:450px;height:250px;">
<caption><center> <u> <font color='purple'> **Рисунок 6** </u><font color='purple'>  : **SIGNS dataset** </center></caption>

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Нормализация
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Конвертирование меток в бинарный вектор
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

In [ ]:
model.fit(X_train, Y_train, epochs = 2, batch_size = 32)

**Ожидаемый выход**:

<table>
    <tr>
        <td>
            ** Epoch 1/2**
        </td>
        <td>
           loss: между 1 и 5, acc: между 0.2 и 0.5.
        </td>
    </tr>
    <tr>
        <td>
            ** Epoch 2/2**
        </td>
        <td>
           loss: между 1 и 5, acc: между 0.2 и 0.5.
        </td>
    </tr>

</table>

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

**Ожидаемый выход**:

<table>
    <tr>
        <td>
            **Test Accuracy**
        </td>
        <td>
           между 0.16 и 0.25
        </td>
    </tr>

</table>

Запустим готовую - обученную специалистами модель

In [ ]:
model = load_model('./pretrain_model/ResNet50.h5') 

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

## 4 - Тестирование со своими изображениями

In [ ]:
img_path = 'images/my_image.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.0
print('Input image shape:', x.shape)
my_image = scipy.misc.imread(img_path)
imshow(my_image)
print("class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = ")
print(model.predict(x))

In [ ]:
model.summary()

In [ ]:
plot_model(model, to_file='model.png')
SVG(model_to_dot(model).create(prog='dot', format='svg'))

## Что вы должны помнить
- Очень глубокие "простые" сети не работают на практике, потому что их трудно обучить из-за исчезающих градиентов. 
- Пропуск-соединения помогают решить проблему исчезающего градиента.
- Существует два основных типа блоков: блок идентификации и сверточный блок. 
- Очень глубокие остаточные сети строятся путем укладки этих блоков вместе.

### Ссылки 

В этом notebook представлен алгоритм ResNet, разработанный He et al. (2015). Реализация приведена в репозитории GitHub Франсуа Шоле:
- Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun - [Deep Residual Learning for Image Recognition (2015)](https://arxiv.org/abs/1512.03385)
- Francois Chollet's GitHub repository: https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py